In [1]:
#setup data
import pandas as pd
import numpy as np
import math
import random
from sklearn import preprocessing

ignore_features = ['25', '26', '8', '58', '20', '23', '29', '31', '32', '35']
toencode = {}
f = open("field_types.txt", 'r')
for line in f.readlines():
    line = line.strip().split(" ")
    if line[0] not in ignore_features:
        toencode[line[0]] = len(line) - 1
        

f1 = open("data.csv")
data_df = pd.read_csv(f1, sep=',', header = 'infer', low_memory=False)
data_df = data_df.drop(ignore_features, axis=1)
label_df = data_df[['label']]
data_df = data_df.drop('label', axis=1)
label = np.array(label_df)


f2 = open("quiz.csv")
quiz_df = pd.read_csv(f2, sep=',', header = 'infer', low_memory=False)
quiz_df = quiz_df.drop(ignore_features, axis=1)


In [2]:
#getting test and train
data_end = np.shape(data_df)[0]
df = data_df.append(quiz_df)
df_trans = pd.get_dummies(df, columns=toencode.keys(), dummy_na=False)
data = np.array(df_trans)

data, quiz = data[:data_end], data[data_end:]

In [3]:
#splitting to features and labels by training and test
def splitdata(data, label):
    n = data.shape[0]
    train_num = int(math.ceil(n*.8))
    indices = random.sample(xrange(n), n)

    training_idx, test_idx = indices[train_num:], indices[:train_num]
    training, test = data[training_idx,:], data[test_idx,:]
    lab, testlabel = label[training_idx,:], label[test_idx,:]
    
    return training, lab, test, testlabel

train, lab, test, testlabel = splitdata(data, label)

In [4]:
#evaluator
def eval(preds, testlabels):
    if len(preds) == len(testlabels):
        error = np.count_nonzero(preds != testlabels) / np.float(len(preds))
        return 1 - error

In [5]:
#unraveling
testlabelnum = np.ravel(testlabel)
labnum = np.ravel(lab)
#testtranslabelnum = np.ravel(testtrans)

In [ ]:
testtranslabelnum = np.ravel(testtrans)

In [12]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

#logistic regression (0.87656)
ylogregtest = logreg.fit(train, lab).predict(test)
print eval(ylogregtest, testlabelnum)

0.88344338228


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
#decision tree 30 depth (0.890)
from sklearn.tree import DecisionTreeClassifier
dectree30 = DecisionTreeClassifier(max_depth=30)

ydectree30test = dectree30.fit(train, lab).predict(test)
print eval(ydectree30test, testlabelnum)

0.891790677047


In [ ]:
#adaboost on dec tree max_dept = 30 (0.9197)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost20 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=30),n_estimators=600, learning_rate=1)

yadaboosttest20 = adaboost20.fit(train, lab).predict(test)
print eval(yadaboosttest20, testlabelnum)

In [ ]:
#adaboost on dec tree max_dept = 30' n_estimators 1000 (0.9190)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost20ne = AdaBoostClassifier(DecisionTreeClassifier(max_depth=30),n_estimators=1000, learning_rate=1)

yadaboosttest20ne = adaboost20ne.fit(train, lab).predict(test)
print eval(yadaboosttest20ne, testlabelnum)

In [10]:
from sklearn.feature_selection import VarianceThreshold
vt = VarianceThreshold(threshold=(.9 * (1 - .9)))
trainvt = vt.fit_transform(train)
testvt = vt.transform(test)

In [11]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

#logistic regression (0.87656)
ylogregtestvt = logreg.fit(trainvt, lab).predict(testvt)
print eval(ylogregtestvt, testlabelnum)

0.815137479058


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier()
clf = clf.fit(train, lab)
model = SelectFromModel(clf, prefit=True)
trainetc = model.transform(train)
print trainetc.shape, train.shape   

/Users/radixold/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


NameError: name 'trainlab' is not defined

In [9]:
trainetc = model.transform(train)
print trainetc.shape, train.shape   
testetc = model.transform(test)
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

#logistic regression (0.87656)
ylogregtestvt = logreg.fit(trainetc, lab).predict(testetc)
print eval(ylogregtestvt, testlabelnum)

(25367, 370) (25367, 2721)
0.877747117375


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
#decision tree 30 depth (0.890)
from sklearn.tree import DecisionTreeClassifier
dectree30 = DecisionTreeClassifier(max_depth=30)

ydectree30testetc = dectree30.fit(trainetc, lab).predict(testetc)
print eval(ydectree30testetc, testlabelnum)

0.891396471864


In [12]:
#adaboost on dec tree max_dept = 30 (0.9197)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost20 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=30),n_estimators=600, learning_rate=1)

yadaboosttest20etc = adaboost20.fit(trainetc, lab).predict(testetc)
print eval(yadaboosttest20etc, testlabelnum)

0.915196609835


/Users/radixold/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#adaboost on dec tree max_dept = 30 (0.9197)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
adaboost20 = AdaBoostClassifier(DecisionTreeClassifier(max_depth=30),n_estimators=600, learning_rate=1)

yadaboosttest20 = adaboost20.fit(train, lab).predict(test)
print eval(yadaboosttest20, testlabelnum)